# Auto Portrait Collector

Collecting portraits of celebrities manually is tedious. This notebook is an attempt to delegate this task to a [browser-use](https://github.com/browser-use/browser-use)ing agent.

## Dependencies

In [1]:
!wget -qO- https://astral.sh/uv/install.sh | sh

downloading uv 0.6.17 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!


In [2]:
!uv venv

Using CPython 3.11.12 interpreter at: /usr/bin/python3
Creating virtual environment at: .venv
Activate with: source .venv/bin/activate


In [3]:
!source .venv/bin/activate

In [4]:
!uv pip install browser-use

Using Python 3.11.12 environment at: /usr
Audited 1 package in 305ms


In [5]:
!uv run playwright install --with-deps

Installing dependencies...
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building depe

## Setup

In [6]:
from typing import List
from langchain_openai import ChatOpenAI
from browser_use import Agent, Browser, BrowserConfig, Controller
from browser_use.browser.context import BrowserContextConfig, BrowserContext
from pydantic import BaseModel

from google.colab import userdata
from dotenv import load_dotenv
import asyncio
import os

In [7]:
openai_api_key = userdata.get('OPENAI_API_KEY')
if openai_api_key:
    os.environ['OPENAI_API_KEY'] = openai_api_key
load_dotenv()

False

In [8]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)

We test if the OpenAI client is operational.

In [9]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="J'adore la programmation.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 31, 'total_tokens': 37, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_90122d973c', 'id': 'chatcmpl-BRQCprxIUsajHsEWbrBtH7bIPXFU9', 'finish_reason': 'stop', 'logprobs': None}, id='run-950aa6fa-9c81-492f-8fb1-9c401004b557-0', usage_metadata={'input_tokens': 31, 'output_tokens': 6, 'total_tokens': 37, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

We define the agent to collect target portraits.

In [10]:
browser = Browser(
    config = BrowserConfig(
        headless=True
    )
)
config = BrowserContextConfig(
    allowed_domains=['pinterest.com'],
)
context = BrowserContext(browser=browser, config=config)

In [11]:
class Image(BaseModel):
	description: str
	url: str

class Images(BaseModel):
	images: List[Image]

controller = Controller(output_model=Images)

In [12]:
async def get_image_urls(target: str) -> Images:
    task = f'Go to https://www.pinterest.com/ideas/ and find at least three good portrait images of {target}. ' \
        'Look for images of the person smiling. For good images get their html src attribute. ' \
        'Dont click on images or follow their links, just get the src attribute of the thumbnail.'

    agent = Agent(
        task=task,
        llm=llm,
        controller=controller,
        browser_context=context,
    )
    history = await agent.run()

    result = history.final_result()

    if result:
        parsed: Images = Images.model_validate_json(result)
        return parsed
    else:
        return None

## Collect

We need to authenticate to access google sheets.

In [13]:
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
from google.auth import default
import gspread
creds, _ = default()
gc = gspread.authorize(creds)

We list all google sheets.

In [14]:
spreadsheet_list = gc.list_spreadsheet_files()
for spreadsheet in spreadsheet_list:
  print(f"Spreadsheet Name: {spreadsheet['name']}, ID: {spreadsheet['id']}")

Spreadsheet Name: Celebrities, ID: 1xB-V6SzTp9mx9FTN7KVvx0QOtfn54U1G4AYumMFJ5V8


We select a sheet and verify it exists and contains targets.

In [15]:
spreadsheet_id = '1xB-V6SzTp9mx9FTN7KVvx0QOtfn54U1G4AYumMFJ5V8'  #@param {type: "string"}
worksheet_name = 'Sheet1' #@param {type: "string"}

try:
  sh = gc.open_by_key(spreadsheet_id)
  worksheet = sh.worksheet(worksheet_name)

  all_values = worksheet.get_all_values()

  for row in all_values[1:]:
      print(row)

except gspread.SpreadsheetNotFound:
  print(f"Spreadsheet with ID '{spreadsheet_id}' not found.")
except gspread.WorksheetNotFound:
  print(f"Worksheet '{worksheet_name}' not found in the spreadsheet.")
except Exception as e:
  print(f"An error occurred: {e}")


['Angourie Rice', '']
['Emma Watson', '']
['Jenna Ortega', '']
['Millie Bobby Brown', '']


We iterate over all rows and start the agent for each. Found images are inserted as columns.

In [16]:
for row_index, row in enumerate(all_values[1:]):
  target = row[0]
  if target:

    print(f'collecting for {target}')
    images = await get_image_urls(target)

    if images is None:
      print(f'no images found')
    else:
      for image_index, image in enumerate(images.images):
        print(f'{image.url}\t{image.description}')
        worksheet.update_cell(row_index + 2, 2 + image_index, f'=IMAGE("{image.url}")')

collecting for Angourie Rice


/usr/local/lib/python3.11/dist-packages/browser_use/agent/service.py:321: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/usr/local/lib/python3.11/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.11/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace

https://i.pinimg.com/236x/c0/aa/fd/c0aafd560cbab191f96a34773f3be9b4.jpg	Image of Angourie Rice smiling
https://i.pinimg.com/236x/d9/f5/83/d9f583c00016ba2db2e3e654e1673c5f.jpg	Image of Angourie Rice smiling
https://i.pinimg.com/236x/19/70/0a/19700acacd16b19b4e371e7530adbde6.jpg	Image of Angourie Rice smiling
collecting for Emma Watson
https://i.pinimg.com/236x/fb/69/21/fb6921c657cb5ffb02db4e48aa0fe2e9.jpg	Emma Watson smiling portrait 1
https://i.pinimg.com/236x/33/ef/6a/33ef6a7bb648655d7b3b08fbd629c31f.jpg	Emma Watson smiling portrait 2
https://i.pinimg.com/236x/7b/d7/bd/7bd7bd58b3703903a1e8c779760097cc.jpg	Emma Watson smiling portrait 3
collecting for Jenna Ortega
https://i.pinimg.com/236x/ae/1e/22/ae1e22eb50e432e69bb0b358331c1bd9.jpg	Jenna Ortega smiling portrait 1
https://i.pinimg.com/236x/30/e0/2d/30e02d4108f52f13e8103960dfd18b01.jpg	Jenna Ortega smiling portrait 2
https://i.pinimg.com/236x/4a/03/8a/4a038ad4c076dc0d6aa4f1fa95c0d6bf.jpg	Jenna Ortega smiling portrait 3
collecting for 

We set the row height to 200px and column width to 150px so that the images are displayed well.

In [17]:
service = build('sheets', 'v4', credentials=creds)
requests = [
    # Update row height
    {
        "updateDimensionProperties": {
            "range": {
                "sheetId": worksheet.id,
                "dimension": "ROWS",
                "startIndex": 1,
                "endIndex": len(all_values)
            },
            "properties": {
                "pixelSize": 200
            },
            "fields": "pixelSize"
        }
    },
    # Update column width
    {
        "updateDimensionProperties": {
            "range": {
                "sheetId": worksheet.id,
                "dimension": "COLUMNS",
                "startIndex": 1,
                "endIndex": 4
            },
            "properties": {
                "pixelSize": 150
            },
            "fields": "pixelSize"
        }
    }
]

body = {
    'requests': requests
}
response = service.spreadsheets().batchUpdate(
    spreadsheetId=spreadsheet_id,
    body=body
).execute()

print("Row height updated successfully!")

Row height updated successfully!
